In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, count, avg, round, udf, corr

In [2]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder\
    .appName("My Spark App") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 09:29:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [4]:
df_client_immat = spark.sql("SELECT * FROM clients_immatriculations")

In [5]:
df_client_immat.printSchema()

root
 |-- immatriculation: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: double (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: boolean (nullable = true)
 |-- taux_eligible: boolean (nullable = true)
 |-- marque: string (nullable = true)
 |-- modele: string (nullable = true)
 |-- puissance: integer (nullable = true)
 |-- longueur: string (nullable = true)
 |-- nbplaces: integer (nullable = true)
 |-- nbportes: integer (nullable = true)
 |-- couleur: string (nullable = true)
 |-- occasion: boolean (nullable = true)
 |-- prix: integer (nullable = true)
 |-- categorie: string (nullable = true)



## Vérification des données :

- Vérification des doublons.

In [6]:
#Compter les doublons
df_duplicates = (
    df_client_immat.groupBy("immatriculation")
    .agg(count("*").alias("count"))
    .filter(col("count") > 1)  # Filtrer les doublons (où le compte est > 1)
)

# Afficher les résultats
df_duplicates.count()

0

- Vérification des nulls.

In [7]:
# Calculer le nombre de nulls pour chaque colonne
null_counts = df_client_immat.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in [
        "immatriculation", "age", "sexe", "taux", "situationfamiliale",
        "nbenfantacharge", "deuxiemevoiture", "taux_eligible", "marque",
        "modele", "puissance", "longueur", "nbplaces", "nbportes",
        "couleur", "occasion", "prix", "categorie"
    ]]
)

# Afficher les nulls par colonne
null_counts.show()

+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+
|immatriculation|age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|taux_eligible|marque|modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|prix|categorie|
+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+
|              0|  0|   0|   0|                 0|              0|              0|            0|     0|     0|        0|       0|       0|       0|      0|       0|   0|        0|
+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+



## Retrait des colonne non nécessaires

In [8]:
columns_to_drop = [
    'immatriculation', 'marque', 'modele', 'puissance', 'longueur',
    'nbplaces', 'nbportes', 'couleur', 'occasion', 'prix'
]
df_client_immat = df_client_immat.drop(*columns_to_drop)

In [9]:
df_client_immat.printSchema()

root
 |-- age: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: double (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: boolean (nullable = true)
 |-- taux_eligible: boolean (nullable = true)
 |-- categorie: string (nullable = true)



## Encodage

In [10]:
indexer_sexe = StringIndexer(inputCol="sexe", outputCol="sexe_index").fit(df_client_immat)
df_client_immat = indexer_sexe.transform(df_client_immat)

encoder_sexe = OneHotEncoder(inputCol="sexe_index", outputCol="sexe_encoded").fit(df_client_immat)
df_client_immat = encoder_sexe.transform(df_client_immat)

indexer_categorie = StringIndexer(inputCol="categorie", outputCol="label").fit(df_client_immat)
df_client_immat = indexer_categorie.transform(df_client_immat)

In [11]:
indexer_situationfamiliale = StringIndexer(inputCol="situationfamiliale", outputCol="situationfamiliale_index").fit(df_client_immat)
df_client_immat = indexer_situationfamiliale.transform(df_client_immat)

encoder_situationfamiliale = OneHotEncoder(inputCol="situationfamiliale_index", outputCol="situationfamiliale_encoded").fit(df_client_immat)
df_client_immat = encoder_situationfamiliale.transform(df_client_immat)

In [12]:
df_client_immat = df_client_immat.drop('situationfamiliale') 
df_client_immat = df_client_immat.drop('situationfamiliale_index')
df_client_immat = df_client_immat.drop('categorie')
df_client_immat = df_client_immat.drop('sexe') 
df_client_immat = df_client_immat.drop('sexe_index') 

df_client_immat = df_client_immat.withColumnRenamed('sexe_encoded', 'sexe')
df_client_immat = df_client_immat.withColumnRenamed('situationfamiliale_encoded', 'situationfamiliale')
df_client_immat = df_client_immat.withColumnRenamed('categorie_indexed', 'categorie')


In [13]:
corr_Matrice = df_client_immat.corr()
sns.heatmap(corr_Matrice, annot=True)
sns.set(rc={"figure.figsize": (30, 16)})
plt.show()

corr_Matrice

TypeError: corr() missing 2 required positional arguments: 'col1' and 'col2'

In [ ]:
df_client_immat = df_client_immat.withColumn(
    "deuxiemevoiture",
    when(col("deuxiemevoiture") == False, 0)
    .when(col("deuxiemevoiture") == True, 1)
    .otherwise(col("deuxiemevoiture").cast("int"))
)

df_client_immat = df_client_immat.withColumn(
    "taux_eligible",
    when(col("taux_eligible") == False, 0)
    .when(col("taux_eligible") == True, 1)
    .otherwise(col("taux_eligible").cast("int"))
)
df_client_immat.show(5)

In [ ]:
#df_client_immat.groupBy("label").count().show()

In [ ]:
# Assemblage des colonnes de caractéristiques
feature_columns = ["age", "taux", "nbenfantacharge", "taux_eligible", "sexe", "situationfamiliale"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(df_client_immat)

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Initialisation du modèle Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")


In [ ]:
# Définition de la grille des hyperparamètres à tester
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [5, 10])  # Exemple de plages pour la profondeur de l'arbre
             .addGrid(dt.minInstancesPerNode, [1, 2, 5])  # Exemple de plages pour le nombre minimum d'instances par noeud
             .addGrid(dt.maxBins, [32, 64, 128])  # Exemple de plages pour le nombre maximal de "bins"
             .build())

In [ ]:
# Évaluateur pour évaluer les performances du modèle
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
# Configuration du CrossValidator
cv = CrossValidator(estimator=dt,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)  # Utilisation de 3 plis pour la validation croisée


In [ ]:
# Entraînement du modèle avec CrossValidation
cv_model = cv.fit(train_data)

In [ ]:
# Prédictions avec le meilleur modèle trouvé
predictions = cv_model.transform(test_data)

# Évaluation de l'accuracy du modèle
accuracy = evaluator.evaluate(predictions)
print(f"Best model accuracy: {accuracy}")

# Évaluation de F1 Score
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = f1_evaluator.evaluate(predictions)
print(f"Best model F1 Score: {f1_score}")

# Affichage des 5 premières lignes des prédictions
predictions.show(5)